### Step 2: Undistort images

Undistorts images needed for estimating the mapping between IR and RGB camera. Provide corresponding IR and RGB images in "images_mapping/ir/*.tiff" and "images_mapping/rgb/*.jpg". Note, that these can be either the same images or different images as provided in the "images_calibration" folder. Undistorted images will be provided in "images_undistorted/ir" and "images_undistorted/rgb".

In [10]:
import os
import glob
import pickle
import cv2
import numpy as np
from tqdm import tqdm

from utils import load_images, resize

In [11]:
root_dir= "camera_30hz"

In [12]:
for mode in ["rgb", "ir"]:

    if mode == "ir":
        images, file_names = load_images(os.path.join(root_dir, "images_calibration", "ir", "*.tiff"), mode, max_num=None, preprocess_ir=False)
        alpha = 1.0  # include all pixels
    elif mode == "rgb":
        images, file_names = load_images(os.path.join(root_dir, "images_calibration", "rgb", "*.jpg"), mode, max_num=None, preprocess_ir=False)
        images = resize(images, 1280, 720)
        alpha = 0.0  # include only valid pixels
        
    if not len(images):
        continue

    output_dir = os.path.join(root_dir, "images_undistorted", mode)
    os.makedirs(output_dir, exist_ok=True)
    
    width, height = images[0].shape[1], images[0].shape[0]
    
    camera_matrix = pickle.load(open(os.path.join(root_dir, "parameters", mode, "camera_matrix.pkl"), "rb"))
    dist_coeffs = pickle.load(open(os.path.join(root_dir, "parameters", mode, "dist_coeffs.pkl"), "rb"))
    
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (width, height), alpha=alpha)
    mapx, mapy = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, None, new_camera_matrix, (width, height), cv2.CV_32FC1)
    pickle.dump(new_camera_matrix, open(os.path.join(root_dir, "parameters", mode, "new_camera_matrix.pkl"), "wb"))
    pickle.dump(mapx, open(os.path.join(root_dir, "parameters", mode, "mapx.pkl"), "wb"))
    pickle.dump(mapy, open(os.path.join(root_dir, "parameters", mode, "mapy.pkl"), "wb"))

    for image, file_name in zip(images, file_names):        
        image_undistorted = cv2.remap(image, mapx, mapy, cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        name = str.split(os.path.basename(file_name), ".")[0]
        if mode == "ir":
            out_file = os.path.join(output_dir, "{}.tiff".format(name))
        elif mode == "rgb":
            out_file = os.path.join(output_dir, "{}.png".format(name))
        cv2.imwrite(out_file, image_undistorted)

100%|██████████| 229/229 [00:00<00:00, 886.29it/s]
